[View in Colaboratory](https://colab.research.google.com/github/keisuke6616/14-DBDA/blob/master/Section3_3.ipynb)

In [0]:
import numpy as np
import numpy.linalg as la
from scipy.stats import norm

import pandas as pd

import matplotlib.pyplot as plt

import time

# Section3.3

## $(\alpha, \beta) = (0.1, 0.1)$

$p=400$ and $p=1200$ 



In [418]:
np.random.seed(42)
t1 = time.time()

N_class = 2
dim = [400, 1200]
N_dim = len(dim)

result = np.zeros((N_dim, N_class, 6))
m = np.zeros((N_dim, N_class))

index = [0, 1]
iter = 2000
c = [0.3, 0.4]
ab = [0.1, 0.1]

for p, pc in zip(dim, index):
    
    print('Dimension : {}'.format(p))
    mu = []
    mu.append(np.r_[np.ones(30), np.zeros(p - 30)])
    mu.append(np.zeros(p))
    Delta_L = la.norm(mu[0] - mu[1])**2

    Sigma, P, Q = [], [], []
    z, trSgm = [], []
    for k in range(N_class):
        S = np.zeros((p, p))
        for i in range(p):
            for j in range(p):
                S[i, j] = (1/2 + (i+1) / (p+1))**(1/2) * c[k]**(np.abs(i - j)**(1/3)) * (1/2 + (j + 1) / (p + 1))**(1/2)
        Sigma.append(S)
        value, vec = la.eig(Sigma[k])
        P.append(np.sqrt(value)); Q.append(vec.T)

        z.append(norm.isf(ab[k], loc=0, scale=1))
        trSgm.append(np.trace(np.dot(Sigma[k], Sigma[k])))

    z_sum = np.sum(z)**2
    C = np.zeros(N_class)
    for k in range(N_class):
        C[k] = z_sum * np.max(trSgm)**(1/2) / (Delta_L**2) * trSgm[k]**(1/4) * np.sum(np.array(trSgm)**(1/4)) + 1
        m[pc, k] = int(np.ceil(0.5 * (C[k] - 1)) + 1)


    N = [[], []]
    e = np.zeros(N_class)
    for l in range(iter):
        x, mean, trS = [], [], []
        W = []
        for k in range(N_class):
            Z = np.array([P[k] * np.random.normal(0, 1, p) for j in range(int(m[pc, k]))])    
            X = np.dot(Z, Q[k]) + mu[k]
            W.append(ECDM(X))

            cov = np.cov(X, rowvar=False)
            trS.append(np.trace(cov))
            mean.append(np.mean(X, axis=0))

            #create new data
            y = P[k] * np.random.normal(0, 1, p)
            x.append(np.dot(y, Q[k]) + mu[k])


        for k in range(N_class):
            w = np.ceil(z_sum * np.max(W)**(1/2) / (Delta_L**2) * W[k]**(1/4) * np.sum(np.array(W)**(1/4))) + 1
            N[k].append(np.max([m[pc, k], w]))

            classifier = np.dot(x[k] - (mean[0] + mean[1]) / 2, mean[1] - mean[0]) - trS[0] / (2 * m[pc, 0]) + trS[1] / (2 * m[pc, 1])
            gamma = Delta_L * (z[0] - z[1]) / (2 * np.sum(z))
            if (-1)**k * (classifier - gamma) < 0:
                e[k] += 1

    N_mean, N_var = [], []
    for k in range(N_class):
        N_mean.append(np.mean(N[k]))
        N_var.append(float(np.cov(N[k])))


    for k in range(N_class):
        result[pc, k, 0] = np.round(C[k], 2)
        result[pc, k, 1] = np.round(N_mean[k], 2)
        result[pc, k, 2] = np.round(N_mean[k] - C[k], 2)
        result[pc, k, 3] = np.round(N_var[k], 2)
        result[pc, k, 4] = np.round(1 - e[k] / iter, 3)
        result[pc, k, 5] = np.round(np.sqrt((1 - e[k] / iter) * (1 - (1 - e[k] / iter)) / iter), 5)

t2 = time.time() - t1
print('Calculation time : {}'.format(t2))

Dimension : 400
Dimension : 1200
Calculation time : 227.63151097297668


In [428]:
print('When (a, b) = (0.1, 0.1)')

print('p=400; (m1, m2) = ({}, {})'.format(int(m[0, 0]), int(m[0, 1])))
pd.DataFrame(result[0], columns=['A', 'B', 'C', 'D', 'E', 'F'], index=[1, 2])

When (a, b) = (0.1, 0.1)
p=400; (m1, m2) = (8, 9)


,A,B,C,D,E,F
1,14.43,14.88,0.46,15.84,0.176,0.00851
2,16.10,16.71,0.61,29.24,0.207,0.00905


In [430]:
print('When (a, b) = (0.1, 0.1)')

print('p=1200; (m1, m2) = ({}, {})'.format(int(m[1, 0]), int(m[1, 1])))
pd.DataFrame(result[1], columns=['A', 'B', 'C', 'D', 'E', 'F'], index=[1, 2])

When (a, b) = (0.1, 0.1)
p=1200; (m1, m2) = (22, 24)


,A,B,C,D,E,F
1,41.72,42.10,0.38,16.75,0.180,0.00858
2,46.92,47.42,0.50,32.52,0.221,0.00927


## $(\alpha, \beta) = (0.05, 0.15)$

In [431]:
np.random.seed(42)
t1 = time.time()

N_class = 2
dim = [400, 1200]
N_dim = len(dim)

result = np.zeros((N_dim, N_class, 6))
m = np.zeros((N_dim, N_class))

index = [0, 1]
iter = 2000
c = [0.3, 0.4]
ab = [0.05, 0.15]

for p, pc in zip(dim, index):
    
    print('Dimension : {}'.format(p))
    mu = []
    mu.append(np.r_[np.ones(30), np.zeros(p - 30)])
    mu.append(np.zeros(p))
    Delta_L = la.norm(mu[0] - mu[1])**2

    Sigma, P, Q = [], [], []
    z, trSgm = [], []
    for k in range(N_class):
        S = np.zeros((p, p))
        for i in range(p):
            for j in range(p):
                S[i, j] = (1/2 + (i+1) / (p+1))**(1/2) * c[k]**(np.abs(i - j)**(1/3)) * (1/2 + (j + 1) / (p + 1))**(1/2)
        Sigma.append(S)
        value, vec = la.eig(Sigma[k])
        P.append(np.sqrt(value)); Q.append(vec.T)

        z.append(norm.isf(ab[k], loc=0, scale=1))
        trSgm.append(np.trace(np.dot(Sigma[k], Sigma[k])))

    z_sum = np.sum(z)**2
    C = np.zeros(N_class)
    for k in range(N_class):
        C[k] = z_sum * np.max(trSgm)**(1/2) / (Delta_L**2) * trSgm[k]**(1/4) * np.sum(np.array(trSgm)**(1/4)) + 1
        m[pc, k] = int(np.ceil(0.5 * (C[k] - 1)) + 1)


    N = [[], []]
    e = np.zeros(N_class)
    for l in range(iter):
        x, mean, trS = [], [], []
        W = []
        for k in range(N_class):
            Z = np.array([P[k] * np.random.normal(0, 1, p) for j in range(int(m[pc, k]))])    
            X = np.dot(Z, Q[k]) + mu[k]
            W.append(ECDM(X))

            cov = np.cov(X, rowvar=False)
            trS.append(np.trace(cov))
            mean.append(np.mean(X, axis=0))

            #create new data
            y = P[k] * np.random.normal(0, 1, p)
            x.append(np.dot(y, Q[k]) + mu[k])


        for k in range(N_class):
            w = np.ceil(z_sum * np.max(W)**(1/2) / (Delta_L**2) * W[k]**(1/4) * np.sum(np.array(W)**(1/4))) + 1
            N[k].append(np.max([m[pc, k], w]))

            classifier = np.dot(x[k] - (mean[0] + mean[1]) / 2, mean[1] - mean[0]) - trS[0] / (2 * m[pc, 0]) + trS[1] / (2 * m[pc, 1])
            gamma = Delta_L * (z[0] - z[1]) / (2 * np.sum(z))
            if (-1)**k * (classifier - gamma) < 0:
                e[k] += 1

    N_mean, N_var = [], []
    for k in range(N_class):
        N_mean.append(np.mean(N[k]))
        N_var.append(float(np.cov(N[k])))


    for k in range(N_class):
        result[pc, k, 0] = np.round(C[k], 2)
        result[pc, k, 1] = np.round(N_mean[k], 2)
        result[pc, k, 2] = np.round(N_mean[k] - C[k], 2)
        result[pc, k, 3] = np.round(N_var[k], 2)
        result[pc, k, 4] = np.round(1 - e[k] / iter, 3)
        result[pc, k, 5] = np.round(np.sqrt((1 - e[k] / iter) * (1 - (1 - e[k] / iter)) / iter), 5)

t2 = time.time() - t1
print('Calculation time : {}'.format(t2))

Dimension : 400
Dimension : 1200
Calculation time : 241.7327914237976


In [432]:
print('When (a, b) = (0.1, 0.1)')

print('p=400; (m1, m2) = ({}, {})'.format(int(m[0, 0]), int(m[0, 1])))
pd.DataFrame(result[0], columns=['A', 'B', 'C', 'D', 'E', 'F'], index=[1, 2])

When (a, b) = (0.1, 0.1)
p=400; (m1, m2) = (9, 10)


,A,B,C,D,E,F
1,15.69,16.11,0.42,14.91,0.125,0.00738
2,17.53,18.09,0.56,29.85,0.246,0.00963


In [433]:
print('When (a, b) = (0.1, 0.1)')

print('p=1200; (m1, m2) = ({}, {})'.format(int(m[1, 0]), int(m[1, 1]))
pd.DataFrame(result[1], columns=['A', 'B', 'C', 'D', 'E', 'F'], index=[1, 2])

SyntaxError: ignored

# Define ECDM

In [0]:
def ECDM(X):
    N, p = X.shape
    
    n = []
    n.append(int(np.ceil(N / 2)))
    n.append(N - n[0])
    
    K = [i for i in range(3, 2*N)]
    index =  [i for i in range(len(K))]

    
    V = [[], []]
    Y = np.zeros((2, len(K), p))
    for k, pc in zip(K, index):
        dv = int(np.floor(k / 2))

        if dv < n[0]:
            V[0].append([i for i in range(dv)] + [i for i in range(dv + n[1], N)])   
        else:
            V[0].append([i for i in range(dv - n[0], dv)])

        if dv <= n[0]:
            V[1].append([i for i in range(dv, dv + n[1])])
        else:
            V[1].append([i for i in range(dv - n[0])] + [i for i in range(dv, N)])
           
        for i in range(2):
            Y[i, pc] = np.sum(X[V[i][pc]], axis=0) / n[i]
            
    w = 0
    for j in range(N):
        for i in range(j):
            w += np.dot(X[i] - Y[0][i + j - 1], X[j] - Y[1][i + j - 1]) ** 2

    u =  n[0] * n[1] / ((n[0] - 1) * (n[1] - 1))
    W = 2 * u / (N * (N - 1)) * w

    return W



    